In [1]:
using Pkg
Pkg.activate(".")
Pkg.add("Distributions")
Pkg.instantiate()
Pkg.precompile()

  Activating project at `/Users/Shared/BEE-4750-Final-Project`
   Resolving package versions...
      Compat entries added for 
  No Changes to `/Users/Shared/BEE-4750-Final-Project/Project.toml`
  No Changes to `/Users/Shared/BEE-4750-Final-Project/Manifest.toml`
  0 dependencies successfully precompiled in 1 seconds. 79 already precompiled.

The following 1 direct dependency failed to precompile:

BEE4750FinalProject 

Error: Missing source file for Base.PkgId(Base.UUID("12345678-1234-1234-1234-123456789012"), "BEE4750FinalProject"  0 dependencies successfully precompiled in 0 seconds. 79 already precompiled.

The following 1 direct dependency failed to precompile:

BEE4750FinalProject 

Error: Missing source file for Base.PkgId(Base.UUID("12345678-1234-1234-1234-123456789012"), "BEE4750FinalProject"  0 dependencies successfully precompiled in 0 seconds. 79 already precompiled.



Base.Precompilation.PkgPrecompileError: The following 1 direct dependency failed to precompile:

BEE4750FinalProject 

Error: Missing source file for Base.PkgId(Base.UUID("12345678-1234-1234-1234-123456789012"), "BEE4750FinalProject"

In [10]:
using JuMP
using HiGHS
using Plots
using Random
Random.seed!(42)

# Data

T = 1:30                                    # time step (days/mo)

Q_data = rand(328767.12:541666.67, 30)      # Data center waste heat [MW] at each t
alpha = 2000                                # Water evaporation factor [L/MWh rejected]

# Greenhouse Daily Thermal Demand Function (30 days)
function monthly_heating_demand_MWh(U::Float64, A::Float64, volume::Float64,
                                    ACH::Float64, Tin::Float64,
                                    Tout_avg::Vector{Float64})
    # Constants
    ρ = 1.2       # kg/m³, air density
    cp = 1005.0   # J/(kg·K), specific heat

    # Compute effective heat-loss coefficient (UA_eff)
    Vdot = ACH * volume / 3600.0      # m³/s
    UA_eff = U * A + ρ * cp * Vdot   # W/°C

    daily_MWh = Float64[]  # output array

    for Tout in Tout_avg
        ΔT = max(Tin - Tout, 0.0)       # Only heat if indoor > outdoor
        Q_W = UA_eff * ΔT                # W
        E_MWh = Q_W * 24.0 / 1e6         # MWh/day
        push!(daily_MWh, E_MWh)
    end

    return daily_MWh
end

# Greenhouse parameters
U = 3.5          # W/m²·K
A = 200.0        # m²
volume = 400.0   # m³
ACH = 1.0        # air changes per hour
Tin = 18.0       # °C, indoor setpoint

# Example: average daily outdoor temps for a month (°C)

Tout_avg_month = rand(0.0:0.1:6.0, 30)

# Compute daily heating demand
D_th = monthly_heating_demand_MWh(U, A, volume, ACH, Tin, Tout_avg_month)

# Model

model = Model(HiGHS.Optimizer)

@variable(model, x_reuse[t in T] >= 0)              # MWh/day reused in CEA
@variable(model, x_reject[t in T] >= 0)             # MWh/day rejected to cooling tower

# Objective
@objective(model, Min, sum(alpha * x_reject[t] for t in T))

# Constraints
@constraint(model, [t in T], x_reuse[t] + x_reject[t] == Q_data[t])
@constraint(model, [t in T], x_reuse[t] <= D_th[t])

optimize!(model)

println("Optimal total water use [L]: ", objective_value(model))
println("\nHour  x_reuse[MWh]  x_reject[MWh]")

for t in T
    println(rpad(t, 5), " ",
            rpad(value(x_reuse[t]), 12), " ",
            rpad(value(x_reject[t]), 12))
end


ArgumentError: ArgumentError: Package Plots not found in current path.
- Run `import Pkg; Pkg.add("Plots")` to install the Plots package.